---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [241]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from adspy_shared_utilities import plot_class_regions_for_classifier
from sklearn.ensemble import RandomForestClassifier


def blight_model():
    
    #df = pd.read_csv("readonly/train.csv", encoding = "ISO-8859-1")
    df = pd.read_csv("train.csv", encoding = "ISO-8859-1")
    #df["prob"]=np.nan
    #print(df.columns)
    df.drop(["payment_amount","payment_date","payment_status","balance_due","collection_status","compliance_detail"],axis=1,inplace=True)
    
    df.set_index("ticket_id", inplace=True)
    
    
    #print(df)
    #print(X_train.prob)
    #print(df.head(2))
    #X_test = pd.read_csv("readonly/test.csv")
    #X_test.drop(["payment_amount","payment_date","payment_status","balance_due","collection_status","compliance_detail","compliance"],axis=1,inplace=True)
    col = ["violation_street_number","mailing_address_str_number","fine_amount","admin_fee","state_fee","late_fee","discount_amount","clean_up_cost", "judgment_amount","compliance"]
    df = df.loc[:,col]
    X_df = df.loc[:,col[0:-2]]
    y_df = df.compliance
    
    

    X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state = 0)
    #print(X_test.head(),y_test.head())
    #clean NaN values
    X_train.mailing_address_str_number.fillna(0,inplace=True)
    X_train.fine_amount.fillna(0,inplace=True)
    y_train.fillna(-1,inplace=True)

    #convert y_test into dataframe
    y_test_df = pd.DataFrame(data=y_test, columns=['compliance'])
    
    #remove NaN values from test set
    X_test.dropna(inplace=True)    
    y_test_df.dropna(inplace=True)
#    test_set = X_test.join(y_test_df)
    test_set = pd.merge(X_test, y_test_df, left_index=True, right_index=True)
#    print(test_set)
    X_test2 = test_set.loc[:,col[0:-2]]
    y_test2 = test_set['compliance']
    
    
    
    #print(X_train)
    #print(X_test,y_test)
    nbclf = GaussianNB().fit(X_train, y_train)
    #print('ticket dataset')
    #print('Accuracy of GaussianNB classifier on training set: {:.2f}'.format(nbclf.score(X_train, y_train)))
    #print('Accuracy of GaussianNB classifier on test set: {:.2f}'.format(nbclf.score(X_test2, y_test2)))
    
    
    clf = RandomForestClassifier(max_features = 2, random_state = 0)
    clf.fit(X_train, y_train)

    #print('Breast cancer dataset')
    #print('Accuracy of RF classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
    #print('Accuracy of RF classifier on test set: {:.2f}'.format(clf.score(X_test2, y_test2)))
    
    
    prediction = nbclf.predict_proba(X_test2)
    #print(len(prediction))
    prediction_df = pd.DataFrame(data=prediction, columns=['Null','0','1'])
#    print(prediction_df)
    
    
    
    
    test_set.reset_index(inplace=True)
    
    result = test_set.join(prediction_df)#pd.merge(df,prediction_df,left_index=True,right_index=True)
    result.set_index('ticket_id',inplace=True)
    #result.dropna(subset=['Null'],inplace=True)
    result = test_set.join(prediction_df)
    
    
    
    
    
    #testdata = pd.read_csv('readonly/test.csv')
    testdata = pd.read_csv('test.csv')
    testdata.set_index("ticket_id", inplace=True)
    
    
    X_testdata = testdata.loc[:,col[0:-2]]
    #X_testdata['compliance']=""
    #X_testdata.convert_objects(convert_numeric=True)
    X_testdata[col[0:-2]] = X_testdata[col[0:-2]].apply(pd.to_numeric, errors='coerce', axis=1)
    #y_testdata = df.compliance
    X_testdata.mailing_address_str_number.fillna(0,inplace=True)
    X_testdata.fine_amount.fillna(0,inplace=True)
    #X_testdata.dropna(inplace=True) 
    #print(X_testdata.head(10000))
    #print(X_testdata.index)
    
    prediction_test = clf.predict_proba(X_testdata)
    prediction_testdf = pd.DataFrame(data=prediction_test, columns=['Null','0','1'])
    X_testdata.reset_index(inplace=True)
    #print(prediction_test)
    
    result_test = X_testdata.join(prediction_testdf)#pd.merge(df,prediction_df,left_index=True,right_index=True)
    result_test.set_index('ticket_id',inplace=True)
    
   
    #return result.loc[:,'1']#df.iloc[:,'predictprob']
    return result_test.loc[:,'1']
#blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Breast cancer dataset
Accuracy of RF classifier on training set: 0.99
Accuracy of RF classifier on test set: 0.93


ticket_id
284932    0.000000
285362    0.000000
285361    0.000000
285338    0.400000
285346    0.000000
285345    0.000000
285347    0.000000
285342    0.500000
285530    0.200000
284989    0.000000
285344    0.000000
285343    0.000000
285340    0.000000
285341    0.000000
285349    0.100000
285348    0.100000
284991    0.000000
285532    0.000000
285406    0.000000
285001    0.000000
285006    0.000000
285405    0.000000
285337    0.000000
285496    0.700000
285497    0.700000
285378    0.000000
285589    0.000000
285585    0.000000
285501    0.000000
285581    0.000000
            ...   
376367    0.000000
376366    0.100000
376362    0.000000
376363    0.100000
376365    0.000000
376364    0.000000
376228    0.475000
376265    0.000000
376286    0.600000
376320    0.000000
376314    0.000000
376327    0.550000
376385    0.100000
376435    0.400000
376370    0.400000
376434    0.014286
376459    0.100000
376478    0.000000
376473    0.000000
376484    0.000000
376482    0.000000
37

In [177]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ticket_id
154739    6.110506e-02
28080     1.262524e-01
132479    1.596803e-01
179377    4.118313e-34
31915     2.002397e-01
176277    2.406423e-01
211593    2.485771e-01
120381    2.203581e-01
153158    1.292348e-01
258831    1.938475e-01
190962    1.274697e-02
187262    2.527240e-01
41224     1.170340e-01
206626    2.566173e-01
141298    1.945498e-01
96460     2.472307e-01
54198     1.131314e-01
106838    1.303797e-01
259615    4.846940e-07
127966    1.744647e-01
212820    1.419774e-01
74602     1.501935e-02
165732    1.352036e-03
65087     1.267693e-01
36465     1.792751e-01
36208     2.150570e-01
88874     2.252950e-01
92608     2.466329e-01
128220    1.463695e-01
178499    2.177920e-01
              ...     
163913    1.048859e-01
222520    7.031525e-34
83791     1.112252e-01
92189     2.321416e-01
273859    1.757869e-01
276091    4.357351e-02
262665    2.055423e-01
205643    2.814623e-17
25363     2.136119e-01
281359    2.622483e-01
124811    4.999267e-34
100576    9.918214e-01
1

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Data type Test: Passed
Data shape Test: Failed: len(bm) should be 61001
Data Values Test: Passed
Data Values type Test: Passed
Index type Test: Passed
Index values type Test: Passed
Output index shape test:Failed, bm.index.shape should be (61001,)
Output index test: Failed
